# Farmer example

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

The implementation directly follows the documentation of StochasticPrograms.jl: https://martinbiel.github.io/StochasticPrograms.jl/stable/manual/examples/

In [ ]:
using StochasticPrograms
using HiGHS

We will add some benchmark tools in order to better evaluate the efficiency of each approach.

In [ ]:
using BenchmarkTools

Using StochasticPrograms.jl's syntax, we describe the two-stage linear program.

In [ ]:
Crops = [:wheat, :corn, :beets]
@stochastic_model farmer_model begin
    @stage 1 begin
        @parameters begin
            Crops = Crops
            Cost = Dict(:wheat=>150, :corn=>230, :beets=>260)
            Budget = 500
        end
        @decision(farmer_model, x[c in Crops] >= 0)
        @objective(farmer_model, Min, sum(Cost[c]*x[c] for c in Crops))
        @constraint(farmer_model, sum(x[c] for c in Crops) <= Budget)
    end
    @stage 2 begin
        @parameters begin
            Crops = Crops
            Required = Dict(:wheat=>200, :corn=>240, :beets=>0)
            PurchasePrice = Dict(:wheat=>238, :corn=>210)
            SellPrice = Dict(:wheat=>170, :corn=>150, :beets=>36, :extra_beets=>10)
        end
        @uncertain ξ[c in Crops]
        @recourse(farmer_model, y[p in setdiff(Crops, [:beets])] >= 0)
        @recourse(farmer_model, w[s in Crops ∪ [:extra_beets]] >= 0)
        @objective(farmer_model, Min, sum(PurchasePrice[p] * y[p] for p in setdiff(Crops, [:beets]))
                   - sum(SellPrice[s] * w[s] for s in Crops ∪ [:extra_beets]))
        @constraint(farmer_model, minimum_requirement[p in setdiff(Crops, [:beets])],
            ξ[p] * x[p] + y[p] - w[p] >= Required[p])
        @constraint(farmer_model, minimum_requirement_beets,
            ξ[:beets] * x[:beets] - w[:beets] - w[:extra_beets] >= Required[:beets])
        @constraint(farmer_model, beets_quota, w[:beets] <= 6000)
    end
end

The package allows to formulate the program as a classical two-stage stochastic program. We now have to provide the scenarios.

In [ ]:
ξ₁ = @scenario ξ[c in Crops] = [3.0, 3.6, 24.0] probability = 1/3
ξ₂ = @scenario ξ[c in Crops] = [2.5, 3.0, 20.0] probability = 1/3
ξ₃ = @scenario ξ[c in Crops] = [2.0, 2.4, 16.0] probability = 1/3

## Deterministic equivalent

We finally pass the complete model to our solver. By default, the deterministic equivalent problem is built.

In [ ]:
farmer = instantiate(farmer_model, [ξ₁,ξ₂,ξ₃], optimizer = HiGHS.Optimizer)

We can check the model formulation.

In [ ]:
println(farmer)

We can now run the optimization solver. We also print the first-stage decision as well as the optimal value.

In [ ]:
optimize!(farmer)
x = optimal_decision(farmer)
x = farmer[1,:x]
println("Wheat: $(value(x[:wheat]))")
println("Corn: $(value(x[:corn]))")
println("Beets: $(value(x[:beets]))")
println("Profit: $(objective_value(farmer))")

The second-stage solutions for the first scenario are

In [ ]:
s = 1  # scenario number
y = farmer[2,:y]
w = farmer[2,:w]
println("Purchased wheat: $(value(y[:wheat], s))")
println("Purchased corn: $(value(y[:corn], s))")
println("Sold wheat: $(value(w[:wheat], s))")
println("Sold corn: $(value(w[:corn], s))")
println("Sold beets: $(value(w[:extra_beets], s))")
println("Profit: $(objective_value(farmer, s))")

`StochasticPrograms` also allows to compute the expected value of perfect information...

In [ ]:
println("EVPI: $(EVPI(farmer))")

... and the value of the stochastic solution.

In [ ]:
println("VSS: $(VSS(farmer))")

## L-shaped decomposition

Instead of the deterministic equivalent, we can use the L-shaped decomposition method.

### Multi-cuts

Let gather the scenario in one vector.

In [ ]:
ξ = [ξ₁, ξ₂, ξ₃]

We instantitate the model with the L-shaped method by specifying it as the chosen optimizer. By default, the multi-cut approach is selected.

In [ ]:
farmer_lshaped = instantiate(farmer_model, ξ, optimizer = LShaped.Optimizer)

The model is now explicitly a two-stage model.

In [ ]:
print(farmer_lshaped)

We have to specify the solver for each stage. This allows to choose more adapted algorithms

In [ ]:
set_optimizer_attribute(farmer_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(farmer_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

We are now in position to solve the program. `StochasticPrograms` informs us on the number of iterations and generated cuts.

In [ ]:
optimize!(farmer_lshaped)

Let measure the performances.

In [ ]:
@benchmark optimize!(farmer_lshaped)

We want to compare it with the simple cut version, so we ask the program to aggregate the cuts.

In [ ]:
set_optimizer_attribute(farmer_lshaped, Aggregator(), Aggregate())

If we solve the problem, we now see that we need more iterations, while a few less cuts are generated. As expected, one cut is now produced at each iteration, except the last one, that established the convergence.

In [ ]:
optimize!(farmer_lshaped)

The benchmark also exhibits that the single cut technique is significantly slower.

In [ ]:
@benchmark optimize!(farmer_lshaped)